In [1]:
import tensorflow as tf

In [1]:
import os
import librosa
import soundfile as sf
from pathlib import Path




ROOT = Path("PARK_DATA")
OUTPUT_ROOT = Path("dataset_5sec")
OUTPUT_ROOT.mkdir(exist_ok=True)

(HC_OUT := OUTPUT_ROOT / "HC").mkdir(exist_ok=True)
(PD_OUT := OUTPUT_ROOT / "PD").mkdir(exist_ok=True)

SEGMENT_LENGTH_SEC = 5
SR = 16000  # YAMNet expects 16 kHz mono

def split_and_save(audio_path, label_folder):
    audio, _ = librosa.load(audio_path, sr=SR, mono=True)
    segment_samples = SEGMENT_LENGTH_SEC * SR
    total_segments = len(audio) // segment_samples
    
    for i in range(total_segments):
        start = i * segment_samples
        end = start + segment_samples
        segment = audio[start:end]
        
        # Only save segments that are exactly 5 seconds
        if len(segment) == segment_samples:
            out_path = (HC_OUT if "HC" in str(label_folder) else PD_OUT) / f"{audio_path.stem}_{i:04d}.wav"
            sf.write(out_path, segment, SR)

# Process all files
for folder in ["HC_segments", "PD_segments"]:
    label_path = ROOT / folder
    for wav_file in label_path.rglob("*.wav"):
        split_and_save(wav_file, label_path)

### Training Yamnet

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from pathlib import Path

# ------------------------------- 1. Load YAMNet (supports batching in 2024+) -------------------------------
yamnet = hub.load('https://tfhub.dev/google/yamnet/1')

--------------------- 4. Extract YAMNet embeddings (batched & fast) -------------------------------

C:\Users\Felix\anaconda3\Lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


NameError: name 'waveforms' is not defined

In [2]:
@tf.function  # This makes it fast and handles single waveforms perfectly
def extract_single_embedding(waveform):
    """Extracts averaged embedding for ONE 5-sec waveform."""
    scores, embeddings, spectrograms = yamnet(waveform)
    return tf.reduce_mean(embeddings, axis=0, keepdims=True)  # (1, 1024)

# ------------------------------- 3. Load your 5-second segments -------------------------------
def load_data(data_dir):
    files = list(Path(data_dir).rglob("*.wav"))
    waveforms = []
    labels = []
    for f in files:
        waveform, _ = librosa.load(f, sr=16000, mono=True)
        # Force exactly 5 seconds (80000 samples) – truncate or pad
        target_len = 80000
        if len(waveform) > target_len:
            waveform = waveform[:target_len]
        elif len(waveform) < target_len:
            waveform = np.pad(waveform, (0, target_len - len(waveform)), mode='constant')
        waveforms.append(waveform)
        labels.append(0 if "HC" in str(f) else 1)  # 0 = Healthy Control, 1 = Parkinson's
    return np.array(waveforms, dtype=np.float32), np.array(labels)  # Ensure float32

print("Loading data...")
waveforms, labels = load_data("dataset_5sec")
print(f"Data loaded successfully: {waveforms.shape[0]} samples, {np.mean(labels):.2%} PD")

Loading data...
Data loaded successfully: 644 samples, 43.63% PD


In [3]:
# ------------------------------- 3. Train/val split -------------------------------
X_train, X_val, y_train, y_val = train_test_split(
    waveforms, labels, test_size=0.2, stratify=labels, random_state=42
)
# ----------

In [4]:
def extract_embeddings_vectorized(waveforms_np, batch_size=32):
    """
    Processes waveforms in batches using tf.vectorized_map for parallel single-sample extraction.
    No shape errors – works on any TF version.
    """
    def map_fn(waveform):
        return extract_single_embedding(waveform)
    
    # Create dataset and process in parallel
    dataset = tf.data.Dataset.from_tensor_slices(waveforms_np)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    all_embeddings = []
    for batch in dataset:
        # vectorized_map processes the batch in parallel (each sample independently)
        batch_emb = tf.vectorized_map(map_fn, batch)  # (batch_size, 1, 1024) → squeeze to (batch_size, 1024)
        all_embeddings.append(tf.squeeze(batch_emb, axis=1).numpy())
    
    return np.concatenate(all_embeddings, axis=0)

print("Extracting YAMNet embeddings (parallel processing – 1-3 min)...")
train_emb = extract_embeddings_vectorized(X_train)
val_emb = extract_embeddings_vectorized(X_val)
print(f"Embeddings ready: train {train_emb.shape}, val {val_emb.shape}")

Extracting YAMNet embeddings (parallel processing – 1-3 min)...


Embeddings ready: train (515, 1024), val (129, 1024)


In [7]:
# ------------------------------- 5. Train classification head -------------------------------
input_layer = tf.keras.layers.Input(shape=(1024,))

x = tf.keras.layers.Dense(512, activation='relu')(input_layer)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)

output = tf.keras.layers.Dense(1, activation='sigmoid')(x)  # PD probability

classifier = tf.keras.Model(input_layer, output)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)
print("Training classifier on YAMNet embeddings...")
history = classifier.fit(
    train_emb, y_train,
    validation_data=(val_emb, y_val),
    epochs=100,
    batch_size=64,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=12, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=6, factor=0.5)
    ],
    verbose=1
)



Training classifier on YAMNet embeddings...
Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.5204 - auc: 0.5328 - loss: 0.7086 - val_accuracy: 0.6202 - val_auc: 0.6737 - val_loss: 0.6609 - learning_rate: 1.0000e-04
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.6019 - auc: 0.5877 - loss: 0.6708 - val_accuracy: 0.6124 - val_auc: 0.6541 - val_loss: 0.6562 - learning_rate: 1.0000e-04
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6330 - auc: 0.6627 - loss: 0.6376 - val_accuracy: 0.6124 - val_auc: 0.6552 - val_loss: 0.6521 - learning_rate: 1.0000e-04
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6466 - auc: 0.6901 - loss: 0.6359 - val_accuracy: 0.6357 - val_auc: 0.6899 - val_loss: 0.6396 - learning_rate: 1.0000e-04
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6252 - auc: 0.6629 - loss: 0.6400 - val_accuracy: 0.6434 - val_auc: 0.7102 - val_loss: 0.6310 - learning_rate: 1.0000e-04
Epoch 6/100
9/9 ━━━━━━━━━━

In [60]:
# ------------------------------- 6. Final evaluation -------------------------------
val_loss, val_acc, val_auc = classifier.evaluate(val_emb, y_val, verbose=0)
print(f"\nValidation Accuracy: {val_acc:.3f} | AUC: {val_auc:.3f}")

# ------------------------------- 7. Save the classifier head (optional) -------------------------------
classifier.save("yamnet_pd_classifier_head.keras")
print("Classifier head saved as 'yamnet_pd_classifier_head.keras'")


Validation Accuracy: 0.791 | AUC: 0.878
Classifier head saved as 'yamnet_pd_classifier_head.keras'


In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# ------------------------------- 5. Train classification head (with smart callbacks) -------------------------------
input_layer = tf.keras.layers.Input(shape=(1024,))

x = tf.keras.layers.Dense(512, activation='relu')(input_layer)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)

output = tf.keras.layers.Dense(1, activation='sigmoid', name='pd_probability')(x)

classifier = tf.keras.Model(input_layer, output)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

print("Training classifier on YAMNet embeddings (smart early stopping + best model saving)...")

# CALLBACKS – This is the gold standard for medical/binary tasks
callbacks = [
    # 1. Stop training when val_auc stops improving (best metric for imbalanced PD data)
    EarlyStopping(
        monitor='val_auc',          # Focus on AUC, not accuracy (more robust for PD detection)
        mode='max',                 # We want to maximize AUC
        patience=15,                # Wait 15 epochs after peak
        restore_best_weights=True,
        verbose=1
    ),
    
    # 2. Reduce learning rate when stuck
    ReduceLROnPlateau(
        monitor='accuracy',
        mode='max',
        factor=0.5,                 # Halve LR
        patience=7,
        min_lr=1e-7,
        verbose=1
    ),
    
    # 3. Save ONLY the best model (by AUC)
    ModelCheckpoint(
        filepath='best_pd_yamnet_classifier.keras',  # or .h5 if older TF
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    )
]

history = classifier.fit(
    train_emb, y_train,
    validation_data=(val_emb, y_val),
    epochs=200,                    # High number – callbacks will stop early
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)



Training classifier on YAMNet embeddings (smart early stopping + best model saving)...
Epoch 1/200
6/9 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5303 - auc: 0.5273 - loss: 0.7174
Epoch 1: val_auc improved from None to 0.74902, saving model to best_pd_yamnet_classifier.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 0.5612 - auc: 0.5327 - loss: 0.7082 - val_accuracy: 0.7829 - val_auc: 0.7490 - val_loss: 0.6645 - learning_rate: 1.0000e-04
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5359 - auc: 0.5409 - loss: 0.6918
Epoch 2: val_auc did not improve from 0.74902
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5689 - auc: 0.5796 - loss: 0.6747 - val_accuracy: 0.6589 - val_auc: 0.7420 - val_loss: 0.6447 - learning_rate: 1.0000e-04
Epoch 3/200
7/9 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6016 - auc: 0.6051 - loss: 0.6701
Epoch 3: val_auc did not improve from 0.74902
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.6117 - auc: 0.6004 - loss: 0.67

In [24]:
# ------------------------------- Final evaluation on best model -------------------------------
print("\n" + "="*60)
print("TRAINING FINISHED – Loading best model and evaluating...")
best_model = tf.keras.models.load_model('best_pd_yamnet_classifier.keras')

val_loss, val_acc, val_auc = best_model.evaluate(val_emb, y_val, verbose=0)
print(f"BEST VALIDATION → Accuracy: {val_acc:.4f} | AUC: {val_auc:.4f}")
print("="*60)




TRAINING FINISHED – Loading best model and evaluating...
BEST VALIDATION → Accuracy: 0.8140 | AUC: 0.8115


In [8]:
val_loss, val_acc, val_auc = classifier.evaluate(val_emb, y_val, verbose=0)
print(f"BEST VALIDATION → Accuracy: {val_acc:.4f} | AUC: {val_auc:.4f}")
print("="*60)

BEST VALIDATION → Accuracy: 0.7907 | AUC: 0.8778


In [14]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

# 1. Load YAMNet as KerasLayer (trainable=False)
yamnet_layer = hub.KerasLayer("https://tfhub.dev/google/yamnet/1", trainable=False)

# 2. Load your trained classifier head
classifier_head = tf.keras.models.load_model("best_pd_yamnet_classifier.keras")
# or: classifier_head = tf.keras.models.load_model("yamnet_pd_classifier_final.keras")

# 3. Create the end-to-end model with tf.map_fn (this bypasses the batching bug)
class PDYamNetEndToEnd(tf.keras.Model):
    def __init__(self, yamnet, head):
        super().__init__()
        self.yamnet = yamnet
        self.head = head

    def call(self, inputs, training=False):
        # inputs: (batch, 80000) or (80000,)
        # Force single-sample processing → no more shape casting error
        def process_one(waveform):
            waveform = tf.expand_dims(waveform, 0)                    # (1, 80000)
            waveform = tf.cast(waveform, tf.float32) / 32768.0        # → [-1, 1]
            _, embeddings, _ = self.yamnet(waveform)                  # (1, time, 1024)
            pooled = tf.reduce_mean(embeddings, axis=1)               # (1, 1024)
            return self.head(pooled)                                  # (1, 1)

        # This handles both batched and single input
        result = tf.map_fn(
            process_one,
            inputs,
            fn_output_signature=tf.float32
        )
        return tf.squeeze(result, axis=-1)  # (batch,) or scalar

# Build the model
model = PDYamNetEndToEnd(yamnet_layer, classifier_head)

# Build with correct shape
model.build(input_shape=(None, 80000))
print('Pipeline succesfully done')

Pipeline succesfully done


In [ ]:
# TEST IT — this will now work!
dummy = np.random.randn(1, 80000).astype(np.float32) * 0.1
pred = model(dummy, training=False)
print("Success! Test prediction:", pred.numpy()[0])   # → something like 0.32

In [ ]:
# 5. Convert to TFLite (fully quantized for maximum speed & smallest size)
converter = tf.lite.TFLiteConverter.from_keras_model(full_model)


# This quantization gives ~4× speed-up and 4× smaller file with almost zero accuracy drop
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]          # Float16 quantization (best for mobile)
converter.inference_input_type = tf.float32                   # You can also use tf.int16 if you prefer
converter.inference_output_type = tf.float32

tflite_model = converter.convert()

# 6. Save the final model
with open("pd_yamnet_end2end.tflite", "wb") as f:
    f.write(tflite_model)

print("SUCCESS! → pd_yamnet_end2end.tflite created")
print(f"Size: {len(tflite_model)/1024/1024:.1f} MB")
print("Ready for Flutter – just drop it into assets/")

In [16]:
# 1. Download the official pre-converted YAMNet TFLite (already quantized & perfect)
import urllib.request
urllib.request.urlretrieve(
    "https://storage.googleapis.com/audioset/yamnet.tflite",
    "yamnet.tflite"
)

# 2. Save ONLY your classifier head as TFLite (this always works)
classifier_head = tf.keras.models.load_model("best_pd_yamnet_classifier.keras")

converter = tf.lite.TFLiteConverter.from_keras_model(classifier_head)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

pd_classifier_tflite = converter.convert()

with open("pd_classifier_only.tflite", "wb") as f:
    f.write(pd_classifier_tflite)

print("Done! Two files created:")
print("   - yamnet.tflite          (15 MB)  ← Google's official")
print("   - pd_classifier_only.tflite (∼2 MB) ← Your PD detector")

INFO:tensorflow:Assets written to: C:\Users\Felix\AppData\Local\Temp\tmpjmeflhxi\assets


INFO:tensorflow:Assets written to: C:\Users\Felix\AppData\Local\Temp\tmpjmeflhxi\assets


Saved artifact at 'C:\Users\Felix\AppData\Local\Temp\tmpjmeflhxi'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1024), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1525379669008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1525379670160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1525379669776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1525379670736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1525379670352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1525379671312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1525379670928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1525379671888: TensorSpec(shape=(), dtype=tf.resource, name=None)
Done! Two files created:
   - yamnet.tflite          (15 MB)  ← Google's official
   - pd_classifier_only.tflite (∼2 MB) ← Your PD detector


In [23]:
import numpy as np
import librosa
import tensorflow as tf

Interpreter = tf.lite.Interpreter

# Load models
yamnet = Interpreter(model_path="yamnet.tflite")
pd     = Interpreter(model_path="pd_classifier_only.tflite")
yamnet.allocate_tensors()
pd.allocate_tensors()

# === CRITICAL: Run once with dummy data to materialize real output shapes ===
dummy_input = np.zeros(15600, dtype=np.float32)
yamnet.set_tensor(yamnet.get_input_details()[0]['index'], dummy_input)
yamnet.invoke()                                   # ← THIS LINE WAS MISSING

# Now detect which output has exactly 1024 elements (the embedding)
output_details = yamnet.get_output_details()
embedding_idx = None
for i, od in enumerate(output_details):
    tensor = yamnet.get_tensor(od['index'])
    if tensor.size == 1024:                       # ← this is the key
        embedding_idx = i
        print(f"Embedding found at output {i} → shape {tensor.shape}")
        break

if embedding_idx is None:
    print("All outputs:")
    for i, od in enumerate(output_details):
        t = yamnet.get_tensor(od['index'])
        print(f"  Output {i}: shape {t.shape} → {t.size} elements")
    raise RuntimeError("Embedding not found")

# === NOW TEST WITH REAL AUDIO ===
audio_path = "dataset_5sec/PD/ID02_pd_2_0_0.wav_seg0_0000.wav"   # ← CHANGE THIS
# audio_path = "dataset_5sec/HC/your_hc_file.wav"

waveform, _ = librosa.load(audio_path, sr=16000, mono=True)
if len(waveform) > 15600:
    waveform = waveform[:15600]
else:
    waveform = np.pad(waveform, (0, 15600 - len(waveform)))
waveform = waveform.astype(np.float32)

# Run YAMNet
yamnet.set_tensor(yamnet.get_input_details()[0]['index'], waveform)
yamnet.invoke()

# Get embedding
embedding = yamnet.get_tensor(output_details[embedding_idx]['index'])  # (1,1024) or (1024,)

# Make sure it's (1, 1024)
if embedding.shape == (1024,):
    embedding = embedding.reshape(1, 1024)

# Run your PD classifier
pd.set_tensor(pd.get_input_details()[0]['index'], embedding)
pd.invoke()
prob = pd.get_tensor(pd.get_output_details()[0]['index'])[0][0]

# Result
label = "Parkinson’s" if prob > 0.5 else "Healthy Control"
conf = max(prob, 1 - prob)

print("\n" + "="*60)
print(f"FILE           : {audio_path.split('/')[-1]}")
print(f"PD PROBABILITY : {prob:.4f}")
print(f"PREDICTION     : {label}")
print(f"CONFIDENCE     : {conf:.1%}")
print("="*60)

All outputs:
  Output 0: shape (1, 521) → 521 elements


RuntimeError: Embedding not found

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import librosa


In [29]:

# --- 1. Configuration (Required by YAMNet) ---
# YAMNet requires audio to be resampled to 16kHz and mono
SAMPLE_RATE = 16000
EMBEDDING_SIZE = 1024 # The dimension of YAMNet's output embedding

# --- 2. Load the Feature Extractor Model ---
# This is the base YAMNet model which extracts the 1024-dim features
# The handle points to the latest YAMNet model on TF Hub
YAMNET_MODEL_HANDLE = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(YAMNET_MODEL_HANDLE)

# --- 3. Function to Process Audio and Extract Embeddings ---
def get_embeddings(audio_path):
    """
    Loads audio, extracts YAMNet embeddings, and returns the average embedding.
    """
    # Load audio, resample to 16kHz, mono, and normalize to [-1.0, 1.0]
    waveform, _ = librosa.load(audio_path, sr=SAMPLE_RATE, mono=True)
    
    # Extract raw embeddings using the YAMNet model
    # The output is (scores, embeddings, spectrogram)
    # The embeddings shape will be (N, 1024), where N is the number of frames
    _, embeddings, _ = yamnet_model(waveform)
    
    # Average the embeddings across all frames (N) to get a single (1024,) vector.
    # This matches the input shape expected by your custom classifier
    # trained on averaged embeddings.
    avg_embedding = np.mean(embeddings, axis=0, keepdims=True) # Shape (1, 1024)
    return avg_embedding

# --- 4. Testing on Dummy Audio ---

# **REPLACE THIS WITH YOUR AUDIO FILE PATH**
dummy_audio_file = 'PARK_DATA/PD_segments/ID20_pd_3_0_1.wav_seg1.wav' 
# **REPLACE THIS WITH YOUR ACTUAL CLASS NAMES**
# This list is necessary to interpret the prediction index
class_names = ['Parkinson_Negative', 'Parkinson_Positive'] 

print("\n" + "="*60)
print(f"Testing on Dummy Audio: **{dummy_audio_file}**")

# Get the averaged YAMNet embedding
test_embedding = get_embeddings(dummy_audio_file)

# Load your custom classifier model
best_model = tf.keras.models.load_model('best_pd_yamnet_classifier.keras')

# Make the prediction
 def predict(folder):
    for files in# The input to `predict` is the embedding with shape (1, 1024)
    predictions = best_model.predict(test_embedding, verbose=0)
    
    # Interpret the Result
    predicted_class_index = np.argmax(predictions[0])
    predicted_probability = predictions[0][predicted_class_index]
    predicted_class_name = class_names[predicted_class_index]
    
    print(f"Raw Prediction (Probabilities): {predictions[0]}")
    print(f"Predicted Class: **{predicted_class_name}**")
    print(f"Confidence: **{predicted_probability:.4f}**")
    print("="*60)


Testing on Dummy Audio: **PARK_DATA/PD_segments/ID20_pd_3_0_1.wav_seg1.wav**
Raw Prediction (Probabilities): [0.6760113]
Predicted Class: **Parkinson_Negative**
Confidence: **0.6760**


In [34]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import librosa
import os # For folder operations
import pandas as pd # For presenting results nicely

# -----------------

In [55]:


# ----------------------------------------------------------------------
## 1. Configuration (Required by YAMNet)
SAMPLE_RATE = 16000
EMBEDDING_SIZE = 1024 
# Set the threshold for classifying the positive class (often 0.5)
CLASSIFICATION_THRESHOLD = 0.5 

# ----------------------------------------------------------------------
## 2. Load the Feature Extractor Model & Custom Classifier
print("Loading YAMNet Feature Extractor...")
YAMNET_MODEL_HANDLE = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(YAMNET_MODEL_HANDLE)

print("Loading Custom Classifier...")
# Load your custom classifier model once
best_model = tf.keras.models.load_model('best_pd_yamnet_classifier.keras')

# ----------------------------------------------------------------------
## 3. Function to Process Audio and Extract Embeddings
def get_embeddings(audio_path):
    """
    Loads audio, extracts YAMNet embeddings, and returns the average embedding.
    """
    try:
        # Load audio, resample to 16kHz, mono
        waveform, _ = librosa.load(audio_path, sr=SAMPLE_RATE, mono=True)
    except Exception as e:
        print(f"ERROR loading {audio_path}: {e}")
        return None
    
    # Extract raw embeddings
    _, embeddings, _ = yamnet_model(waveform)
    
    # Average the embeddings across all frames to get a single (1, 1024) vector.
    avg_embedding = np.mean(embeddings, axis=0, keepdims=True)
    return avg_embedding

# ----------------------------------------------------------------------
## 4. Function to Predict for all Files in a Folder
def predict_folder_old(folder_path, class_names):
    """
    Iterates through all .wav files in a folder, extracts features, and predicts
    using a model trained with binary_crossentropy (sigmoid output).
    """
    if not os.path.isdir(folder_path):
        print(f"\n🚫 Error: Folder not found at {folder_path}")
        return

    print("\n" + "="*80)
    print(f"✨ Starting Prediction for folder: **{folder_path}**")
    print("="*80)

    results = []
    
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(('.wav', '.mp3', '.flac')):
            audio_path = os.path.join(folder_path, filename)
            print(f"-> Processing {filename}...")

            test_embedding = get_embeddings(audio_path)
            
            if test_embedding is not None:
                # Make the prediction
                # Output shape is (1, 1) or (1,), representing P(Positive Class)
                predictions = best_model.predict(test_embedding, verbose=0)
                
                # --- ADJUSTMENT FOR BINARY CROSSENTROPY / SIGMOID ---
                
                # The single prediction value is the probability of the positive class
                positive_prob = predictions[0][0] 
                
                # Classify based on the threshold (e.g., 0.5)
                if positive_prob >= CLASSIFICATION_THRESHOLD:
                    predicted_class_name = class_names[1] # The positive class
                    confidence = positive_prob
                else:
                    predicted_class_name = class_names[0] # The negative class
                    # Confidence in the negative class is 1 - positive_prob
                    confidence = 1.0 - positive_prob
                
                # ------------------------------------------------------
                
                # Store the result
                results.append({
                    'Filename': filename,
                    'Predicted_Class': predicted_class_name,
                    'Confidence': f"{confidence:.4f}",
                    'Raw_Positive_Probability': f"{positive_prob:.4f}"
                })
            
    # Display results in a nice table
    if results:
        df = pd.DataFrame(results)
        print("\n" + "-"*80)
        print("✅ Prediction Summary:")
        # Display the filename, predicted class, and the raw positive probability
        print(df[['Filename', 'Predicted_Class', 'Raw_Positive_Probability']].to_markdown(index=False))
        print(f"\nNote: Prediction uses a threshold of **{CLASSIFICATION_THRESHOLD}** on Raw Positive Probability.")
        print("-"*80)
    else:
        print("\n⚠️ No audio files processed.")

# ----------------------------------------------------------------------


Loading YAMNet Feature Extractor...
Loading Custom Classifier...


In [58]:
## 5. Execution Block
# **DEFINE THE FOLDER YOU WANT TO TEST**
TEST_FOLDER = 'PARK_DATA - Copy/PD_segments' 

# **DEFINE YOUR ACTUAL CLASS NAMES**
# Class Names MUST be ordered as [Negative Class, Positive Class]
CLASS_NAMES = ['Parkinson_Negative', 'Parkinson_Positive'] 

predict_folder_old(TEST_FOLDER, CLASS_NAMES)


✨ Starting Prediction for folder: **PARK_DATA - Copy/PD_segments**
-> Processing ID02_pd_2_0_0.wav_seg1.wav...
-> Processing ID04_pd_2_0_1.wav_seg0.wav...
-> Processing ID06_pd_3_1_1.wav_seg0.wav...
-> Processing ID07_pd_2_0_0.wav_seg0.wav...
-> Processing ID13_pd_3_2_2.wav_seg0.wav...
-> Processing ID16_pd_2_0_0.wav_seg0.wav...
-> Processing ID18_pd_4_3_3.wav_seg0.wav...
-> Processing ID20_pd_3_0_1.wav_seg1.wav...
-> Processing ID24_pd_2_0_0.wav_seg1.wav...
-> Processing ID27_pd_4_1_1.wav_seg0.wav...
-> Processing ID29_pd_3_1_2.wav_seg2.wav...
-> Processing ID30_pd_2_1_1.wav_seg1.wav...
-> Processing ID32_pd_3_1_1.wav_seg0.wav...
-> Processing ID33_pd_3_2_2.wav_seg0.wav...
-> Processing ID34_pd_2_0_0.wav_seg1.wav...

--------------------------------------------------------------------------------
✅ Prediction Summary:
| Filename                   | Predicted_Class    |   Raw_Positive_Probability |
|:---------------------------|:-------------------|---------------------------:|
| ID02

### Maximum voting approach

In [67]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import librosa
import os
import pandas as pd
from collections import Counter # For max voting

# ----------------------------------------------------------------------
## 1. Configuration & Global Variables
# YAMNet requirements
SAMPLE_RATE = 16000
EMBEDDING_SIZE = 1024 
# The duration of the segment the model was trained on
SEGMENT_DURATION_SECONDS = 5 
# Set the threshold for classifying the positive class (often 0.5)
CLASSIFICATION_THRESHOLD = 0.5 

# ----------------------------------------------------------------------
## 2. Load the Feature Extractor Model & Custom Classifier
print("Loading YAMNet Feature Extractor...")
YAMNET_MODEL_HANDLE = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(YAMNET_MODEL_HANDLE)

print("Loading Custom Classifier...")
# Load your custom classifier model once
best_model = tf.keras.models.load_model('best_pd_yamnet_classifier.keras')

# ----------------------------------------------------------------------
## 3. Core Function: Embeddings Extraction
def extract_single_embedding(waveform_segment):
    """
    Extracts the averaged YAMNet embedding from a pre-loaded waveform segment.
    Input: (N,) numpy array of audio samples
    Output: (1, 1024) numpy array
    """
    # Extract raw embeddings using the YAMNet model
    # YAMNet handles the internal 0.96s frames
    _, embeddings, _ = yamnet_model(waveform_segment)
    
    # Average the embeddings across all frames to get a single (1, 1024) vector.
    avg_embedding = np.mean(embeddings, axis=0, keepdims=True)
    return avg_embedding

# ----------------------------------------------------------------------
## 4. Main Function: Segmentation, Prediction, and Voting
def segment_and_predict(audio_path, class_names):
    """
    Loads a long audio file, splits it into 5-second segments,
    predicts the class for each segment, and uses Max Voting for the final result.
    """
    # --- Step 4.1: Load the full audio file ---
    try:
        # Load audio, resample to 16kHz, mono
        full_waveform, sr = librosa.load(audio_path, sr=SAMPLE_RATE, mono=True)
    except Exception as e:
        print(f"ERROR loading {audio_path}: {e}")
        return None, None
    
    total_samples = len(full_waveform)
    segment_samples = SEGMENT_DURATION_SECONDS * sr
    
    if total_samples < segment_samples:
         print(f"Warning: File too short ({total_samples/sr:.2f}s). Skipping segmentation.")
         # If too short, just process it as a single segment
         num_segments = 1
    else:
         # Calculate the number of non-overlapping 5-second segments
         num_segments = total_samples // segment_samples
         
    segment_predictions = []
    
    # --- Step 4.2: Iterate through segments ---
    for i in range(num_segments):
        start_sample = i * segment_samples
        end_sample = start_sample + segment_samples
        
        # Extract the current segment waveform
        segment_waveform = full_waveform[start_sample:end_sample]

        # Check if the segment is actually 5 seconds long (handles the last segment potentially being shorter if total length isn't an exact multiple)
        if len(segment_waveform) < segment_samples and i != num_segments - 1:
            # Should only happen if the loop condition is wrong, but a safeguard
            continue 
            
        # Extract the 1024-dim embedding
        segment_embedding = extract_single_embedding(segment_waveform)
        
        # Predict the probability of the positive class for the segment
        # predictions shape is (1, 1) -> get the single probability value
        segment_prediction_prob = best_model.predict(segment_embedding, verbose=0)[0][0]

        # Classify the segment based on the threshold
        if segment_prediction_prob >= CLASSIFICATION_THRESHOLD:
            segment_class = class_names[1] # Positive class
        else:
            segment_class = class_names[0] # Negative class
            
        segment_predictions.append(segment_class)
        
    # --- Step 4.3: Maximum Voting ---
    if not segment_predictions:
        return class_names[0], 0, 0, 0 # Default to negative if something went wrong

    # Use Counter to find the most frequent class
    vote_counts = Counter(segment_predictions)
    final_class = vote_counts.most_common(1)[0][0]
    
    # Calculate voting metrics
    total_votes = len(segment_predictions)
    positive_votes = vote_counts.get(class_names[1], 0)
    negative_votes = vote_counts.get(class_names[0], 0)
    
    # The confidence is defined by the proportion of the winning votes
    confidence_score = vote_counts[final_class] / total_votes
    
    return final_class, confidence_score, positive_votes, total_votes

# ----------------------------------------------------------------------
## 5. Execution Block: Iterate Folder
def predict_folder(folder_path, class_names):
    """
    Iterates through all .wav files in a folder and applies the max voting prediction.
    """
    if not os.path.isdir(folder_path):
        print(f"\n🚫 Error: Folder not found at {folder_path}")
        return

    print("\n" + "="*80)
    print(f"✨ Starting Prediction for folder: **{folder_path}** (5s Segments + Max Voting)")
    print("="*80)

    results = []
    
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(('.wav', '.mp3', '.flac', '.m4a')):
            audio_path = os.path.join(folder_path, filename)
            
            # Run the segmentation and voting prediction
            final_class, confidence, positive_votes, total_votes = segment_and_predict(audio_path, class_names)
            
            if final_class:
                results.append({
                    'Filename': filename,
                    'Total_Duration_s': f"{librosa.get_duration(path=audio_path):.2f}",
                    'Total_Segments': total_votes,
                    f'{class_names[1]}_Votes': positive_votes,
                    'Final_Predicted_Class': final_class,
                    'Max_Vote_Confidence': f"{confidence:.4f}",
                })
            
    # Display results in a nice table
    if results:
        df = pd.DataFrame(results)
        print("\n" + "-"*120)
        print("✅ Prediction Summary (Segmented by 5s):")
        # Display the result columns
        print(df.to_markdown(index=False))
        print("\nNote: The 'Max_Vote_Confidence' is the proportion of votes for the winning class.")
        print("-"*120)
        final_counts = Counter(df['Final_Predicted_Class'])
        
        print("\n📊 **Final Prediction Tally Across All Files**")
        total_files = len(df)
        print(f"Total files processed: **{total_files}**\n")
        
        for class_name in class_names:
            count = final_counts.get(class_name, 0)
            percentage = (count / total_files) * 100 if total_files > 0 else 0
            print(f"   → **{class_name}**: {count} files ({percentage:.1f}%)")
        
        print("="*60)
    else:
        print("\n⚠️ No audio files processed.")


# ----------------------------------------------------------------------
## 6. Execution Call


Loading YAMNet Feature Extractor...
Loading Custom Classifier...


In [72]:
# **DEFINE THE FOLDER YOU WANT TO TEST**
TEST_FOLDER = 'PARK_DATA/Mine' 

# **DEFINE YOUR ACTUAL CLASS NAMES**
# Class Names MUST be ordered as [Negative Class, Positive Class]
CLASS_NAMES = ['Parkinson_Negative', 'Parkinson_Positive'] 

predict_folder(TEST_FOLDER, CLASS_NAMES)


✨ Starting Prediction for folder: **PARK_DATA/Mine** (5s Segments + Max Voting)

------------------------------------------------------------------------------------------------------------------------
✅ Prediction Summary (Segmented by 5s):
| Filename      |   Total_Duration_s |   Total_Segments |   Parkinson_Positive_Votes | Final_Predicted_Class   |   Max_Vote_Confidence |
|:--------------|-------------------:|-----------------:|---------------------------:|:------------------------|----------------------:|
| Voice 022.wav |              59.3  |               11 |                          8 | Parkinson_Positive      |                0.7273 |
| Voice 023.wav |              50.9  |               10 |                          7 | Parkinson_Positive      |                0.7    |
| Voice 024.wav |              51.04 |               10 |                          4 | Parkinson_Negative      |                0.6    |

Note: The 'Max_Vote_Confidence' is the proportion of votes for the winn

### Threshold Analysis for the best 100 epoch model

In [63]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import balanced_accuracy_score, roc_curve

# --- Placeholder/Simulation for the Keras Model and Data ---
# Since I cannot load your actual model or data, this section simulates the necessary
# objects. If you run this in your environment, replace this with your actual loading code.

# 1. Load the actual model head (Assuming it was saved as 'yamnet_pd_classifier_head.keras')
try:
    classifier = tf.keras.models.load_model("yamnet_pd_classifier_head.keras")
    print("Successfully loaded model: yamnet_pd_classifier_head.keras")
except Exception as e:
    print(f"Could not load the actual model: {e}")
    print("Using a placeholder model and dummy data for demonstration.")
    
    # Define a simple placeholder model (only for simulation if loading fails)
    class DummyClassifier(tf.keras.Model):
        def predict(self, emb, **kwargs):
            # Simulate probabilities for a binary classification task
            num_samples = emb.shape[0]
            # Generate random probabilities heavily skewed around 0.3 for demonstration
            predictions = np.random.beta(a=1, b=3, size=(num_samples, 1))
            return predictions

    # Use the dummy model
    classifier = DummyClassifier()
    
    # 2. Simulate validation data (Replace with your actual data)
    num_samples = 500
    embedding_dim = 1024 # YAMNet output
    val_emb = np.random.rand(num_samples, embedding_dim).astype(np.float32)
    
    # Simulate true labels (y_val), with some imbalance (e.g., 80% negative, 20% positive)
    y_val = np.random.choice([0, 1], size=num_samples, p=[0.8, 0.2]).reshape(-1, 1)


def find_optimal_threshold(y_true, y_proba):
    """
    Finds the optimal threshold for binary classification based on Youden's J statistic
    (which maximizes the difference between True Positive Rate (Sensitivity) and False Positive Rate).
    This often results in a better-balanced threshold than maximizing pure accuracy.

    Args:
        y_true (np.array): True binary labels (0 or 1).
        y_proba (np.array): Predicted probabilities for the positive class (shape N, 1).

    Returns:
        tuple: (best_threshold, best_balanced_accuracy)
    """
    
    # Reshape probabilities if necessary (from N, 1 to N)
    if y_proba.ndim > 1 and y_proba.shape[1] == 1:
        y_proba = y_proba.flatten()

    # Calculate ROC curve metrics (False Positive Rate, True Positive Rate, Thresholds)
    # y_true must be 1D, y_proba must be 1D
    fpr, tpr, thresholds = roc_curve(y_true.flatten(), y_proba)
    
    # Youden's J statistic (Maximize Sensitivity + Specificity - 1)
    # Sensitivity = TPR
    # Specificity = 1 - FPR
    youden_j = tpr - fpr
    
    # Find the index of the best threshold
    optimal_idx = np.argmax(youden_j)
    
    # Get the optimal threshold
    optimal_threshold = thresholds[optimal_idx]

    # Calculate the Balanced Accuracy at this optimal threshold
    y_pred_at_threshold = (y_proba >= optimal_threshold).astype(int)
    best_balanced_acc = balanced_accuracy_score(y_true.flatten(), y_pred_at_threshold)
    
    # We also calculate the standard accuracy at this threshold for comparison
    standard_acc = np.mean(y_true.flatten() == y_pred_at_threshold)
    
    return optimal_threshold, best_balanced_acc, standard_acc


# ------------------------------- 7. Threshold Optimization ------------------------------------

# 1. Get predicted probabilities from the model
print("\nGenerating predictions on validation set...")
# The .predict() output is typically a numpy array of probabilities
y_pred_proba = classifier.predict(val_emb)


# 2. Find the optimal threshold
best_threshold, balanced_acc, standard_acc = find_optimal_threshold(y_val, y_pred_proba)

# 3. Print the results
print("\n--- Optimal Threshold Analysis ---")
print(f"Total Validation Samples: {len(y_val)}")
print(f"Positive Samples (1): {np.sum(y_val)}")
print(f"Negative Samples (0): {len(y_val) - np.sum(y_val)}")

print(f"\nBest Classification Threshold: {best_threshold:.4f}")
print(f"Accuracy at this Threshold: {standard_acc:.4f}")
print(f"Balanced Accuracy (Optimal Metric): {balanced_acc:.4f}")

# You can now use this best_threshold (e.g., 0.38) for all future binary classifications:
# final_predictions = (y_pred_proba >= best_threshold).astype(int)

# Optional: Re-calculate and print AUC for completeness (as you did before)
try:
    val_loss, val_acc, val_auc = classifier.evaluate(val_emb, y_val, verbose=0)
    print(f"\nModel AUC (original, threshold-independent): {val_auc:.4f}")
except:
    # If the dummy model is used, Keras evaluate won't work easily
    pass

print("\nBest threshold check complete.")

Successfully loaded model: yamnet_pd_classifier_head.keras

Generating predictions on validation set...
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

--- Optimal Threshold Analysis ---
Total Validation Samples: 129
Positive Samples (1): 56
Negative Samples (0): 73

Best Classification Threshold: 0.4173
Accuracy at this Threshold: 0.8295
Balanced Accuracy (Optimal Metric): 0.8202

Model AUC (original, threshold-independent): 0.8778

Best threshold check complete.


### COMBINING YAMNET AND CLASSIFIER

In [73]:
import tensorflow as tf
import numpy as np

# --- Configuration ---
CLASSIFIER_HEAD_PATH = "best_pd_yamnet_classifier.keras"
OUTPUT_TFLITE_PATH = "best_pd_yamnet_classifier.tflite"

# YAMNet requires a specific input shape for the embedding features
YAMNET_EMBEDDING_DIM = 1024

def combine_and_convert_model():
    """
    Combines the YAMNet feature extractor (represented by its input shape) 
    with the custom classifier head, converts the result to TFLite, and tests inference.
    """
    print("--- Phase 1: Combining Models ---")

    # 1. Load the custom classifier head
    try:
        classifier_head = tf.keras.models.load_model(CLASSIFIER_HEAD_PATH)
        print(f"Loaded custom classifier head from: {CLASSIFIER_HEAD_PATH}")
    except Exception as e:
        print(f"Error loading classifier head: {e}")
        print("Creating a dummy classifier head for demonstration.")
        # Create a placeholder classifier head for testing the script structure
        classifier_head = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(YAMNET_EMBEDDING_DIM,)),
            tf.keras.layers.Dense(1, activation='sigmoid', name='final_output')
        ])

    # 2. Define the Input Layer for the full model
    # The input to the full model *after* feature extraction is the YAMNet embedding (1024 features)
    # The actual YAMNet model (audio -> embedding) is often deployed separately or included 
    # as a pre-processing step in the final mobile pipeline. For Keras combining, we start 
    # from the embedding layer.
    embedding_input = tf.keras.Input(shape=(YAMNET_EMBEDDING_DIM,), name='yamnet_embedding_input')

    # 3. Connect the input to the classifier head
    final_output = classifier_head(embedding_input)

    # 4. Create the full Keras model
    full_model = tf.keras.Model(inputs=embedding_input, outputs=final_output, name='full_pd_yamnet_classifier')
    full_model.summary()

    print("\n--- Phase 2: Converting to TFLite ---")

    # Use the TFLite converter
    converter = tf.lite.TFLiteConverter.from_keras_model(full_model)

    # Optimization is crucial for mobile devices
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    try:
        tflite_model = converter.convert()
        
        # Save the TFLite model file
        with open(OUTPUT_TFLITE_PATH, "wb") as f:
            f.write(tflite_model)
        
        print(f"✅ Successfully converted and saved TFLite model to: {OUTPUT_TFLITE_PATH}")

        # 5. Test the TFLite model inference
        test_tflite_model(tflite_model)

    except Exception as e:
        print(f"❌ TFLite Conversion Failed: {e}")


In [74]:
def test_tflite_model(tflite_model):
    """
    Tests the TFLite model by performing inference on a dummy input array.
    This verifies that the model is runnable and has the correct input/output shape.
    """
    print("\n--- Phase 3: Testing TFLite Inference ---")
    
    # Initialize the TFLite Interpreter
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()

    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Create a dummy input embedding (simulating one 0.96s audio segment)
    input_shape = input_details[0]['shape'] # Should be (1, 1024)
    # Ensure the input is float32, which is standard for Keras/TFLite
    dummy_input = np.random.rand(*input_shape).astype(np.float32)

    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], dummy_input)

    # Run inference
    interpreter.invoke()

    # Get the output tensor (the prediction probability)
    tflite_output = interpreter.get_tensor(output_details[0]['index'])

    print(f"TFLite Input Shape: {input_details[0]['shape']}")
    print(f"TFLite Output Shape: {output_details[0]['shape']}")
    print(f"TFLite Output Value (Probability): {tflite_output[0][0]:.4f}")

    if tflite_output.shape == (1, 1) and tflite_output.dtype == np.float32:
        print("✅ TFLite model test successful! Ready for mobile integration.")
    else:
        print("❌ TFLite model test failed: Output shape or type is incorrect.")


if __name__ == '__main__':
    combine_and_convert_model()

--- Phase 1: Combining Models ---
Loaded custom classifier head from: best_pd_yamnet_classifier.keras


Model: "full_pd_yamnet_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ yamnet_embedding_input (InputLayer)  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ functional (Functional)              │ (None, 1)                   │         689,153 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 689,153 (2.63 MB)

 Trainable params: 689,153 (2.63 MB)

 Non-trainable params: 0 (0.00 B)


--- Phase 2: Converting to TFLite ---
INFO:tensorflow:Assets written to: C:\Users\Felix\AppData\Local\Temp\tmp4iztsgr_\assets


INFO:tensorflow:Assets written to: C:\Users\Felix\AppData\Local\Temp\tmp4iztsgr_\assets


Saved artifact at 'C:\Users\Felix\AppData\Local\Temp\tmp4iztsgr_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1024), dtype=tf.float32, name='yamnet_embedding_input')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1525360982544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1525360982736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1525360982928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1525863909520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1527316836432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1527316831824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1527316832592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1527316830096: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Successfully converted and saved TFLite model to: best_pd_yamnet_classifier.tflite

--- Phase 3: Testing TFLite Inference ---
TFLite Input

C:\Users\Felix\anaconda3\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [90]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import librosa
import os
import pandas as pd
from collections import Counter
from typing import List, Tuple, Dict, Any

# =================================================================================
# ⚠️ 1. Configuration Constants (MUST MATCH YOUR TRAINING SETUP)
# =================================================================================

# Path to the TFLite model generated in the previous step
TFLITE_MODEL_PATH = "best_pd_yamnet_classifier.tflite"

# The duration of the segment the model was trained on
SEGMENT_DURATION_SECONDS = 5

# YAMNet/Audio Requirements
YAMNET_MODEL_HANDLE = 'https://tfhub.dev/google/yamnet/1' 
TARGET_SAMPLE_RATE = 16000 
YAMNET_EMBEDDING_DIM = 1024

# IMPORTANT: Replace this with the optimal threshold you found previously (e.g., 0.3800)
OPTIMAL_THRESHOLD = 0.49 

# Define class names for output reporting
CLASS_NAMES = ["Healthy (H)", "Parkinson's Disease (PD)"]

# =================================================================================
# ⚠️ 2. Jupyter Setup: Define your audio folder path here
# =================================================================================

# Set the path to the folder containing your test audio files (WAV, MP3, etc.)
# EXAMPLE: AUDIO_FOLDER_PATH = './my_test_audio_data/'
AUDIO_FOLDER_PATH = 'PARK_DATA/PD_segments' 

# =================================================================================
# 3. Core Functions
# =================================================================================

def load_yamnet_feature_extractor():
    """Loads the YAMNet model from TensorFlow Hub to generate embeddings."""
    try:
        yamnet_model = hub.load(YAMNET_MODEL_HANDLE)
        print("YAMNet feature extractor loaded successfully from TensorFlow Hub.")
        return yamnet_model
    except Exception as e:
        print(f"Error loading YAMNet feature extractor: {e}")
        return None

def run_tflite_inference(tflite_model_content: bytes, embedding: np.ndarray) -> float:
    """
    Performs inference on a single (1, 1024) embedding using the TFLite interpreter.
    Returns the prediction probability (float).
    """
    interpreter = tf.lite.Interpreter(model_content=tflite_model_content)
    interpreter.allocate_tensors()
    
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    
    # Input must be shaped as (1, 1024) and be of type float32
    embedding = embedding.astype(np.float32)
    
    interpreter.set_tensor(input_details['index'], embedding)
    interpreter.invoke()
    
    # Get the single prediction probability
    probability = interpreter.get_tensor(output_details['index'])[0][0]
    return probability

def extract_single_embedding(yamnet_model: tf.Module, waveform_segment: tf.Tensor) -> np.ndarray:
    """
    Extracts the YAMNet embedding from a waveform segment and averages it.
    Input: (N,) Tensor of audio samples (e.g., 5 seconds of 16kHz audio)
    Output: (1, 1024) numpy array
    """
    # YAMNet handles the internal 0.96s frames and returns multiple embeddings
    # embeddings shape: (Number_of_sub_frames, 1024)
    _, embeddings, _ = yamnet_model(waveform_segment)
    
    # Average the embeddings across all sub-frames to get a single (1, 1024) vector.
    avg_embedding = np.mean(embeddings.numpy(), axis=0, keepdims=True)
    return avg_embedding

def segment_and_predict(audio_path: str, yamnet_model: tf.Module, tflite_model_content: bytes) -> Tuple[str, float, int, int]:
    """
    Loads a long audio file, splits it into 5-second segments,
    predicts the class for each segment using the averaged embedding, 
    and uses Max Voting for the final result.
    Returns: (final_class, confidence_score, positive_votes, total_votes)
    """
    print(f"\n--- Analyzing: {os.path.basename(audio_path)} ---")
    
    # --- Step 1: Load the full audio file ---
    try:
        # Load audio, resample to 16kHz, mono
        full_waveform, sr = librosa.load(audio_path, sr=TARGET_SAMPLE_RATE, mono=True)
        full_waveform_tf = tf.constant(full_waveform, dtype=tf.float32)
    except Exception as e:
        print(f"ERROR loading {audio_path}: {e}")
        return CLASS_NAMES[0], 0.0, 0, 0
    
    total_samples = len(full_waveform)
    segment_samples = SEGMENT_DURATION_SECONDS * sr
    
    # Calculate number of non-overlapping segments
    num_segments = total_samples // segment_samples
    
    if num_segments == 0:
        # If the file is shorter than 5 seconds, treat the whole file as one segment
        num_segments = 1
        # No need to truncate, full_waveform is used below
    elif total_samples % segment_samples != 0:
        # Ensure we only process full 5-second segments by truncating the end
        full_waveform_tf = full_waveform_tf[:num_segments * segment_samples]

    segment_predictions_label: List[str] = []
    segment_prediction_probs: List[float] = []

    # --- Step 2: Iterate through segments ---
    for i in range(num_segments):
        
        # Get the segment waveform (5s chunk)
        if num_segments == 1 and total_samples < segment_samples:
            # Case 1: File < 5s. segment_waveform is the full waveform
            segment_waveform = full_waveform_tf
        else:
            # Case 2: File >= 5s. Slice the 5s chunk.
            start_sample = i * segment_samples
            end_sample = start_sample + segment_samples
            segment_waveform = full_waveform_tf[start_sample:end_sample]

        # Extract the single averaged 1024-dim embedding for this 5s chunk
        segment_embedding = extract_single_embedding(yamnet_model, segment_waveform)
        
        # Predict the probability of the positive class for the segment using the TFLite model
        segment_prediction_prob = run_tflite_inference(tflite_model_content, segment_embedding)
        segment_prediction_probs.append(segment_prediction_prob)

        # Classify the segment based on the optimal threshold
        if segment_prediction_prob >= OPTIMAL_THRESHOLD:
            segment_class = CLASS_NAMES[1] # Positive class (PD)
        else:
            segment_class = CLASS_NAMES[0] # Negative class (Healthy)
            
        segment_predictions_label.append(segment_class)
    
    print(f"Total audio duration: {total_samples/sr:.2f}s, processed as {num_segments} segment(s).")
    print(f"Segment prediction probabilities: {np.array(segment_prediction_probs).round(4)}")


    # --- Step 3: Maximum Voting ---
    if not segment_predictions_label:
        return CLASS_NAMES[0], 0.0, 0, 0

    # Use Counter to find the most frequent class
    vote_counts = Counter(segment_predictions_label)
    final_class = vote_counts.most_common(1)[0][0]
    
    # Calculate voting metrics
    total_votes = len(segment_predictions_label)
    positive_votes = vote_counts.get(CLASS_NAMES[1], 0)
    
    # The confidence is defined by the proportion of the winning votes
    confidence_score = vote_counts[final_class] / total_votes
    
    return final_class, confidence_score, positive_votes, total_votes

def predict_folder(folder_path: str, yamnet_model: tf.Module, tflite_model_content: bytes):
    """
    Iterates through all supported audio files in a folder and applies the max voting prediction.
    """
    if not os.path.isdir(folder_path):
        print(f"\n🚫 Error: Folder not found at {folder_path}")
        return

    print("\n" + "="*90)
    print(f"✨ Starting Prediction for folder: **{folder_path}** (5s Averaged Embeddings + Max Voting)")
    print(f"Using Optimal Threshold: {OPTIMAL_THRESHOLD:.4f}")
    print("="*90)

    results: List[Dict[str, Any]] = []
    
    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith(('.wav', '.mp3', '.flac', '.m4a')):
            audio_path = os.path.join(folder_path, filename)
            
            # Run the segmentation and voting prediction
            final_class, confidence, positive_votes, total_votes = segment_and_predict(
                audio_path, yamnet_model, tflite_model_content
            )
            
            if final_class:
                try:
                    duration = librosa.get_duration(path=audio_path)
                except Exception:
                    duration = 0.0

                results.append({
                    'Filename': filename,
                    'Duration (s)': f"{duration:.2f}",
                    'Total Segments': total_votes,
                    f'{CLASS_NAMES[1].split(" ")[0]}_Votes': positive_votes,
                    'Final Predicted Class': final_class,
                    'Max Vote Confidence': f"{confidence:.4f}",
                })

    # Display results in a nice table
    if results:
        df = pd.DataFrame(results)
        print("\n" + "-"*120)
        print("✅ Prediction Summary (5s Averaging + Max Voting):")
        
        # Display the result columns
        print(df.to_markdown(index=False))
        print("\nNote: 'Max Vote Confidence' is the proportion of votes for the winning class.")
        print("-"*120)
        final_counts = Counter(df['Final Predicted Class'])
        
        print("\n📊 **Overall Prediction Tally Across All Files**")
        total_files = len(df)
        print(f"Total files processed: **{total_files}**\n")
        
        for class_name in CLASS_NAMES:
            count = final_counts.get(class_name, 0)
            percentage = (count / total_files) * 100 if total_files > 0 else 0
            print(f"    → **{class_name}**: {count} files ({percentage:.1f}%)")
        
        print("="*60)
    else:
        print("\n⚠️ No supported audio files processed.")

def run_prediction_test(folder_path: str):
    """Main execution function to run the prediction pipeline."""
    try:
        # Load TFLite model content once
        with open(TFLITE_MODEL_PATH, "rb") as f:
            tflite_model_content = f.read()
    except FileNotFoundError:
        print(f"Error: TFLite model not found at {TFLITE_MODEL_PATH}. Ensure the file exists.")
        return
        
    # Load the YAMNet feature extractor once
    yamnet_model = load_yamnet_feature_extractor()
    if yamnet_model is None:
        return

    # --- Run Prediction ---
    predict_folder(folder_path, yamnet_model, tflite_model_content)

# --------------------------------------------------------------------------------
# ⚠️ 4. EXECUTION CELL
# 
# To run this in Jupyter, ensure the TFLITE_MODEL_PATH and AUDIO_FOLDER_PATH
# variables are set correctly above, and then execute this line:
# --------------------------------------------------------------------------------

# run_prediction_test(AUDIO_FOLDER_PATH)

In [91]:
run_prediction_test(AUDIO_FOLDER_PATH)

YAMNet feature extractor loaded successfully from TensorFlow Hub.

✨ Starting Prediction for folder: **PARK_DATA/PD_segments** (5s Averaged Embeddings + Max Voting)
Using Optimal Threshold: 0.4900

--- Analyzing: ID02_pd_2_0_0.wav_seg0.wav ---
Total audio duration: 78.78s, processed as 15 segment(s).
Segment prediction probabilities: [0.4443 0.4585 0.5436 0.4717 0.4571 0.7162 0.5074 0.5442 0.5049 0.442
 0.4236 0.5278 0.5318 0.45   0.5222]

--- Analyzing: ID02_pd_2_0_0.wav_seg1.wav ---
Total audio duration: 43.16s, processed as 8 segment(s).
Segment prediction probabilities: [0.5757 0.4943 0.5521 0.5108 0.5461 0.8296 0.4915 0.4888]

--- Analyzing: ID04_pd_2_0_1.wav_seg0.wav ---
Total audio duration: 52.01s, processed as 10 segment(s).
Segment prediction probabilities: [0.4686 0.5379 0.5094 0.5296 0.5117 0.5289 0.6236 0.4664 0.5095 0.47  ]

--- Analyzing: ID04_pd_2_0_1.wav_seg1.wav ---
Total audio duration: 27.85s, processed as 5 segment(s).
Segment prediction probabilities: [0.5509 0.48

In [92]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

# 1. Load the full YAMNet model
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1') 
# The loaded object is a callable that returns (scores, embeddings, spectrogram)

# 2. Identify the embedding output function
def yamnet_embedding_extractor(waveform):
    # Call the model with the 16kHz waveform input
    scores, embeddings, spectrogram = yamnet_model(waveform)
    
    # We only care about the embedding tensor (index 1)
    return embeddings

# 3. Define the input signature for TFLite conversion
# Input shape must be None for dynamic length audio, or a specific shape 
# if you want a fixed-size buffer. Here we use None for flexibility.
# YAMNet input is 1D float32 waveform (16000 samples per second)
concrete_func = tf.function(yamnet_embedding_extractor,
                            input_signature=[
                                tf.TensorSpec(shape=[None], dtype=tf.float32)
                            ])
concrete_func = concrete_func.get_concrete_function()


# 4. Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func], 
                                                          yamnet_model.signatures['serving_default'])
# Note: You may need to set specific optimizations for TFLite conversion here.

tflite_model = converter.convert()

# 5. Save the TFLite file
with open('yamnet_feature_extractor.tflite', 'wb') as f:
    f.write(tflite_model)

print("Successfully created yamnet_feature_extractor.tflite")

Successfully created yamnet_feature_extractor.tflite


In [8]:
import tensorflow as tf
import tensorflow_hub as hub

class Yamnet5sEmbedding(tf.Module):
    def __init__(self):
        super().__init__()
        # Track YAMNet as an attribute
        self.yamnet = hub.load("https://tfhub.dev/google/yamnet/1")

    @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.float32)])
    def embed(self, waveform):
        """
        waveform: 1D tensor, exactly 5 seconds (80,000 samples)
        returns: 1x1024 tensor embedding
        """
        # Use the tracked YAMNet
        scores, embeddings, _ = self.yamnet(waveform)
        pooled = tf.reduce_mean(embeddings, axis=0)  # (1024,)
        pooled = tf.expand_dims(pooled, axis=0)      # (1, 1024)
        return {"embedding": pooled}

# Create module
module = Yamnet5sEmbedding()

# Save as SavedModel
tf.saved_model.save(
    module,
    "yamnet_5s_embed_model",
    signatures={"serving_default": module.embed}
)

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model("yamnet_5s_embed_model")
tflite_model = converter.convert()

# Save TFLite
with open("yamnet_5s_mean_embedding.tflite", "wb") as f:
    f.write(tflite_model)

print("Saved TFLite model: yamnet_5s_mean_embedding.tflite")


INFO:tensorflow:Assets written to: yamnet_5s_embed_model\assets


INFO:tensorflow:Assets written to: yamnet_5s_embed_model\assets


Saved TFLite model: yamnet_5s_mean_embedding.tflite


In [15]:
import numpy as np
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path="yamnet_5s_mean_embedding.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Dummy 5-second audio (batch size 1)
dummy_audio = np.random.randn(1, 80000).astype(np.float32)  # shape (1, 80000)

interpreter.set_tensor(input_details[0]['index'], dummy_audio)
interpreter.invoke()

embedding = interpreter.get_tensor(output_details[0]['index'])
print("Output shape:", embedding.shape)  # (1, 1024)
print("First 5 elements:", embedding[0, :5])


Output shape: (1, 1024)
First 5 elements: [0.         0.         0.00182058 0.08442353 0.        ]


In [13]:
import tensorflow as tf
import tensorflow_hub as hub

class Yamnet5sEmbedding(tf.Module):
    def __init__(self):
        super().__init__()
        self.yamnet = hub.load("https://tfhub.dev/google/yamnet/1")

    @tf.function(input_signature=[tf.TensorSpec(shape=[1, 80000], dtype=tf.float32)])
    def embed(self, waveform):
        """
        waveform: [1, 80000] batch of 1 audio clip
        returns: 1x1024 tensor embedding
        """
        # Remove batch dim for YAMNet
        waveform_1d = tf.squeeze(waveform, axis=0)  # shape (80000,)
        scores, embeddings, _ = self.yamnet(waveform_1d)
        pooled = tf.reduce_mean(embeddings, axis=0)
        pooled = tf.expand_dims(pooled, axis=0)    # (1, 1024)
        return {"embedding": pooled}

# Save SavedModel
module = Yamnet5sEmbedding()
tf.saved_model.save(module, "yamnet_5s_embed_model", signatures={"serving_default": module.embed})

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model("yamnet_5s_embed_model")
tflite_model = converter.convert()

with open("yamnet_5s_mean_embedding.tflite", "wb") as f:
    f.write(tflite_model)

print("Saved TFLite model with 2D input (1, 80000)")


INFO:tensorflow:Assets written to: yamnet_5s_embed_model\assets


INFO:tensorflow:Assets written to: yamnet_5s_embed_model\assets


Saved TFLite model with 2D input (1, 80000)


In [19]:
import numpy as np
import tensorflow as tf
import soundfile as sf  # pip install soundfile

# --- 1. Load 5-second audio file ---
audio_file = "dataset_5sec/HC/ID00_hc_0_0_0.wav_seg0_0000.wav"

# Load waveform, convert to float32, ensure 16 kHz
waveform, sr = sf.read(audio_file, dtype='float32')

# Resample if needed
if sr != 16000:
    import librosa
    waveform = librosa.resample(waveform, orig_sr=sr, target_sr=16000)
    sr = 16000

# Ensure mono
if waveform.ndim > 1:
    waveform = np.mean(waveform, axis=1)

# Check length: must be 5 seconds → 80000 samples
if len(waveform) != 16000*5:
    raise ValueError(f"Audio length is {len(waveform)/sr:.2f}s, must be 5s")

# Add batch dimension for TFLite
waveform = np.expand_dims(waveform, axis=0)  # shape (1, 80000)

# --- 2. Load TFLite model ---
interpreter = tf.lite.Interpreter(model_path="yamnet_5s_mean_embedding.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# --- 3. Run inference ---
interpreter.set_tensor(input_details[0]['index'], waveform)
interpreter.invoke()

# --- 4. Get embedding ---
embedding = interpreter.get_tensor(output_details[0]['index'])
print("Output shape:", embedding.shape)  # should be (1, 1024)
print("First 5 elements:", embedding[0, :5])


Output shape: (1, 1024)
First 5 elements: [0.0792807  0.24947849 0.1258863  0.06407533 0.06327682]
